# Setup

In [10]:
## Profiling Packages
import cProfile
import pstats
import snakeviz
import pickle as pkl

In [7]:
import os, sys
print(os.getcwd())
sys.path.append(os.getcwd()+"/scripts_model")

print(sys.path)

/Users/aidan.mcloughlin/Library/CloudStorage/OneDrive-Veracyte,Inc/Desktop/work/dmPC
['/Users/aidan.mcloughlin/Library/CloudStorage/OneDrive-Veracyte,Inc/Desktop/work/dmPC/scripts_model/profiling', '/Users/aidan.mcloughlin/miniconda3/envs/drug_resp/lib/python3.9/site-packages/git/ext/gitdb', '/Users/aidan.mcloughlin/miniconda3/envs/drug_resp/lib/python39.zip', '/Users/aidan.mcloughlin/miniconda3/envs/drug_resp/lib/python3.9', '/Users/aidan.mcloughlin/miniconda3/envs/drug_resp/lib/python3.9/lib-dynload', '', '/Users/aidan.mcloughlin/miniconda3/envs/drug_resp/lib/python3.9/site-packages', '/Users/aidan.mcloughlin/Library/CloudStorage/OneDrive-Veracyte,Inc/Desktop/work/dmPC/scripts_model']


In [8]:
## Setup code from 2_2 cluster simulation
from setup_simu2_2 import *


pytorch version: 2.0.1
orig num threads: 8
Cancer type coding map: 
  C_type  code  count
0   grp0     0     62
4   grp1     1     35
8   grp2     2     41
Cancer type coding map: 
  C_type  code  count
0   grp0     0     62
3   grp1     1     76
Training data: 
   cdr: (117, 30)
   c_data: (117, 355) 
   d_data: (30, 150)
   Number of each initial cancer clusters: 

code    0   1  Total
code                
0      54   0     54
1       0  30     30
2       0  33     33
Total  54  63    117

Testing data:  
   cdr: (21, 30)
   c_data: (21, 355) 
   d_data: (30, 150)
   Number of each initial cancer clusters: 

code   0   1  Total
code               
0      8   0      8
1      0   5      5
2      0   8      8
Total  8  13     21
cpu


# Train Model

We can see that the brunt of run time is contained within each round of VAE+predictor tuning.  The run time is not noticeably changing between rounds.  We would not expect the run times of these to be so long given how quickly the VAEs are pretraining.

In [9]:
CDPmodel = CDPmodel(K, CDPmodel_args)
n_rounds = 5
fit_returns = CDPmodel.fit(c_train, c_meta_train, d_data, cdr_train, train_args, n_rounds=n_rounds, search_subcluster=True, device = device)
c_meta, c_meta_hist, d_sens_hist, losses_train_hist_list, best_epos_list, C_VAE_init_losses, D_VAE_init_losses, c_latent_list, d_latent_list = fit_returns

=> Initialize C-VAE:
        Best epoc with test loss: epoch 78
        Running time: 0.37354612350463867
=> Initialize D-VAE:
        Best epoc with test loss: epoch 52
        Running time: 0.1433250904083252
########################################################
#### 0. k = 0                                     
########################################################
  === 0.1. Training local CDP model 
     -- round 0 -------------
       a. Training D_VAE and Predictor
            Best epoc with test loss: epoch 5
            Running time: 11.744307041168213
       b. 10 sensitive drug(s)
       c. Training C_VAE and Predictor
            Best epoc with test loss: epoch 9
            Running time: 8.398390054702759
       d. 54 cancer cell line(s) in the cluster
     -- round 1 -------------
       a. Training D_VAE and Predictor
            Best epoc with test loss: epoch 6
            Running time: 11.73641324043274
       b. 10 sensitive drug(s)
       c. Training C_VAE and 

We used VSCode debug mode to collect the environment state during the round 0 local model training functions are invoked.  We load them here:

In [14]:
with open("scripts_model/profiling/output/d_vae_model_dict.pkl", 'rb') as f:
    d_vae_model_dict = pkl.load(f)

with open("scripts_model/profiling/output/c_vae_model_dict.pkl", 'rb') as f:
    c_vae_model_dict = pkl.load(f)

print(pd.DataFrame(d_vae_model_dict.keys()))

                            0
0                       model
1                data_loaders
2               c_names_k_old
3               d_names_k_old
4           C_VAE_loss_weight
5         C_recon_loss_weight
6                C_kld_weight
7   C_cluster_distance_weight
8       C_update_ratio_weight
9           D_VAE_loss_weight
10        D_recon_loss_weight
11               D_kld_weight
12  D_cluster_distance_weight
13      D_update_ratio_weight
14        predict_loss_weight
15                sens_cutoff
16                  optimizer
17                   n_epochs
18                  scheduler
19                  save_path


In [95]:
## Data-Loading Times:
print(1.044 / 3434 * 1000)
print(0.048 / 500 * 1000)

0.30401863715783345
0.096


## TODO list for profiling:
* find an efficient re-write of d_overlap_ratio and c_overlap_ratio
* there appear to be redundant batches. debug why this is happening.
* pre-compute encodings in each round will speed up the model
* compare the result if we set a very large batch size to the model.
* I'll look into literature for gene importance scores with respect to VAE latent space (w/ clusters?).
* Prototype learning for compounds? 

In [70]:
c_vae_model_dict['sens_cutoff'] = 0.5
train_cvae_w_args()

setup is done.
NUMBER OF BATCHES: 18
BATCH SIZE: 50
BATCH DIMENSIONS: 
torch.Size([50, 355])
torch.Size([50, 150])
NUMBER OF BATCHES: 5
BATCH SIZE: 50
BATCH DIMENSIONS: 
torch.Size([50, 355])
torch.Size([50, 150])
            Best epoc with test loss: epoch 7
NUMBER OF CALL TO DRUG SENS CUTOFF FNCT: 0


In [90]:
cProfile.run('train_dvae_w_args()', str(root) + "/scripts_model/profiling/output/dvae_state.prof")


setup is done.
NUMBER OF BATCHES: 25
BATCH SIZE: 50
BATCH DIMENSIONS: 
torch.Size([50, 355])
torch.Size([50, 150])
NUMBER OF BATCHES: 7
BATCH SIZE: 50
BATCH DIMENSIONS: 
torch.Size([50, 355])
torch.Size([50, 150])
            Best epoc with test loss: epoch 9
NUMBER OF CALL TO DRUG SENS CUTOFF FNCT: 0


In [91]:
p = pstats.Stats(str(root) + "/scripts_model/profiling/output/dvae_state.prof")
p.strip_dirs().sort_stats(2).print_stats(75)

Sun Nov 26 21:59:43 2023    /Users/aidan.mcloughlin/Library/CloudStorage/OneDrive-Veracyte,Inc/Desktop/work/dmPC/scripts_model/profiling/output/dvae_state.prof

         5452206 function calls (5253774 primitive calls) in 10.216 seconds

   Ordered by: cumulative time
   List reduced from 341 to 75 due to restriction <75>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   10.216   10.216 {built-in method builtins.exec}
        1    0.000    0.000   10.216   10.216 <string>:1(<module>)
        1    0.000    0.000   10.216   10.216 35102567.py:317(train_dvae_w_args)
        1    1.499    1.499   10.215   10.215 35102567.py:2(train_CDPmodel_local)
153600/9600    0.121    0.000    2.800    0.000 module.py:1494(_call_impl)
     3200    0.010    0.000    2.697    0.001 models.py:718(forward)
     6400    0.011    0.000    2.409    0.000 models.py:823(forward)
41600/28800    0.052    0.000    2.151    0.000 container.py:215(forward)
     640

## VAE pretrainer profiling:

In [52]:
## Forward time comparisons (VAE vs Full Model):
print(.114 / 300)
print(2.490 / 3200)

0.00038
0.0007781250000000001


In [49]:
with open("scripts_model/profiling/output/c_vae_pt_dict.pkl", 'rb') as f:
    c_vae_pt_dict = pkl.load(f)

with open("scripts_model/profiling/output/d_vae_pt_dict.pkl", 'rb') as f:
    d_vae_pt_dict = pkl.load(f)
#     "vae_model": self.CDPmodel_list[0].c_VAE, 
#     "device": device, 
#     "c_data": c_data, 
#     "vae_type": "C", 
#     "save_path": train_params['cVAE_save_path'], 
#     "params": train_params
def train_cvae_w_args():
    train_VAE(
        c_vae_pt_dict['vae_model'], 
        c_vae_pt_dict['device'], 
        c_vae_pt_dict['c_data'], 
        c_vae_pt_dict['vae_type'], 
        c_vae_pt_dict['save_path'],
        c_vae_pt_dict['params'])
        

## Profile:
cProfile.run('train_cvae_w_args()', str(root) + "/scripts_model/profiling/output/vae_pretrain.prof")

p = pstats.Stats(str(root) + "/scripts_model/profiling/output/vae_pretrain.prof")
p.strip_dirs().sort_stats(2).print_stats(50)


        Best epoc with test loss: epoch 99
        Running time: 0.4088881015777588
Sun Nov 26 14:11:56 2023    /Users/aidan.mcloughlin/Library/CloudStorage/OneDrive-Veracyte,Inc/Desktop/work/dmPC/scripts_model/profiling/output/vae_pretrain.prof

         307402 function calls (296061 primitive calls) in 0.410 seconds

   Ordered by: cumulative time
   List reduced from 525 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.410    0.410 {built-in method builtins.exec}
        1    0.000    0.000    0.410    0.410 <string>:1(<module>)
        1    0.000    0.000    0.410    0.410 2494660929.py:12(train_cvae_w_args)
        1    0.000    0.000    0.410    0.410 trainers.py:748(train_VAE)
        1    0.018    0.018    0.409    0.409 trainers.py:636(train_VAE_train)
 4500/600    0.005    0.000    0.121    0.000 module.py:1494(_call_impl)
      300    0.001    0.000    0.114    0.000 models.py:823(forward)

In [ ]:
print(c_vae_pt_dict['params'].batch_size)
print(c_vae_pt_dict['c_data'].shape)


In [89]:
## The original local training function is below:
def train_CDPmodel_local(
        model, 
        data_loaders={}, 
        c_names_k_old=None, 
        d_names_k_old=None, 
        C_VAE_loss_weight = 1, 
        C_recon_loss_weight = 1, 
        C_kld_weight = None, 
        C_cluster_distance_weight=100, 
        C_update_ratio_weight = 100, 
        D_VAE_loss_weight = 1, 
        D_recon_loss_weight = 1, 
        D_kld_weight = None, 
        D_cluster_distance_weight=100, 
        D_update_ratio_weight = 100, 
        predict_loss_weight = 1, 
        sens_cutoff = 0.5,
        optimizer=None, 
        n_epochs=100, 
        scheduler=None,
        load=False, 
        save_path="model.pkl",  
        best_model_cache = "drive"):
    
    call_counter = 0

    if(load!=False):
        if(os.path.exists(save_path)):
            model.load_state_dict(torch.load(save_path))           
            return model, 0, 0
        else:
            logging.warning("Failed to load existing model file, proceed to the trainning process.")
    
    dataset_sizes = {x: data_loaders[x].dataset.tensors[0].shape[0] for x in ['train', 'val']}
    loss_hist = {}
    c_vae_loss_hist = {}
    c_recon_loss_hist = {}
    c_kld_hist = {}
    c_cluster_dist_hist = {}
    c_update_overlap_hist = {}
    d_vae_loss_hist = {}
    d_recon_loss_hist = {}
    d_kld_hist = {}
    d_cluster_dist_hist = {}
    d_update_overlap_hist = {}
    prediction_loss_hist = {}

    n_cells = len(np.unique(np.concatenate(
        (data_loaders['train'].dataset.tensors[3].numpy(),
        data_loaders['val'].dataset.tensors[3].numpy()))))
    n_drugs = len(np.unique(np.concatenate(
        (data_loaders['train'].dataset.tensors[4].numpy(),
        data_loaders['val'].dataset.tensors[4].numpy()))))
    
    if best_model_cache == "memory":
        best_model_wts = copy.deepcopy(model.state_dict())
    else:
        torch.save(model.state_dict(), save_path+"_bestcahce.pkl")
    
    best_loss = np.inf
    best_epoch = -1

    print("setup is done.")

    for epoch in range(n_epochs):
        logging.info('Epoch {}/{}'.format(epoch, n_epochs - 1))
        logging.info('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                # optimizer = scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_prediction_loss = 0.0
            running_c_update_overlap = 0.0
            running_d_update_overlap = 0.0

            running_c_vae_loss = 0.0
            running_c_recon_loss = 0.0
            running_c_kld_loss = 0.0
            running_c_cluster_d = 0.0
            
            running_d_vae_loss = 0.0
            running_d_recon_loss = 0.0
            running_d_kld_loss = 0.0
            running_d_cluster_d = 0.0
            

            n_iters = len(data_loaders[phase])
            batch_size = data_loaders[phase].batch_size


            # Iterate over data.
            # for data in data_loaders[phase]:
            n_batches = 0
            if epoch == 0:
                for batchidx, (y, c_data, d_data, c_name, d_name) in enumerate(data_loaders[phase]):
                    n_batches += 1
            
                print("NUMBER OF BATCHES: " + str(n_batches))
                print("BATCH SIZE: " + str(data_loaders[phase].batch_size)) 

            for batchidx, (y, c_data, d_data, c_name, d_name) in enumerate(data_loaders[phase]):

                if epoch == 0 and batchidx == 0:
                    print("BATCH DIMENSIONS: ")
                    print(c_data.shape)
                    print(d_data.shape)

                
                # if c_data.shape[0] <= 2:
                #     print(f'epoch: {epoch}, phase: {phase}, batch: {batchidx}')
                #     print(f'c_data shape: {c_data.shape}')
                #     print(f'd_data shape: {d_data.shape}')
                
                y.requires_grad_(True)
                d_data.requires_grad_(True)

                # encode and decode 
                c_mu, c_log_var, c_X_rec, d_mu, d_log_var, d_X_rec, y_hat = model(c_data, d_data)

                sensitive = y_hat > sens_cutoff
                sensitive = sensitive.long()

                # compute loss

                mse = nn.MSELoss(reduction="sum")

                #   1. Prediction loss: 
                bce = nn.BCELoss()
                prediction_loss = bce(y_hat, y)
                
                if C_VAE_loss_weight > 0:
                    # 2. C_VAE:
                    # 2.1. VAE loss: reconstruction loss & kld
                    C_recon_loss, C_kld = custom_vae_loss(c_data, c_mu, c_log_var, c_X_rec, mse)

                    # 2.2. the loss of latent spaces distances:
                    #     - distances of cells in the cluster to the cluster centroid 
                    #     + distances of cells outside the cluster to the cluster centroid 
                    C_latent_dist_loss = cluster_mu_distance(c_mu, sensitive)
                    
                    # adding all up
                    if C_kld_weight is None:
                        C_kld_weight = batch_size / dataset_sizes[phase]

                    C_VAE_loss = C_recon_loss_weight * C_recon_loss + C_kld_weight * C_kld + C_cluster_distance_weight * C_latent_dist_loss # + C_update_ratio_weight * C_overlap_loss


                    # 3. requiring the updated cluster overlaping with the old clustering
                    c_name_dt = c_name.detach().numpy().astype(int)
                    c_name_uq = np.sort(np.unique(c_name_dt))
                    c_sens_k = torch.zeros(n_cells)
                    for c in c_name_uq:
                        c_sens_k[c] = torch.mean(y_hat[c_name_dt == c]) > sens_cutoff

                    if len(set(c_names_k_old)) > 0:
                        c_overlap_ratio = torch.sum(c_sens_k[c_names_k_old]) / sum([c in c_name_uq for c in c_names_k_old])
                        C_overlap_loss = -c_overlap_ratio # at least 50% overlapping, the more overlap the better?
                    else:
                        C_overlap_loss = 0
                
                else:
                    C_VAE_loss = 0
                    C_overlap_loss = 0


                
                if D_VAE_loss_weight > 0:
                    # 4. D_VAE
                    # 4.1. VAE loss: reconstruction loss & kld
                    D_recon_loss, D_kld = custom_vae_loss(d_data, d_mu, d_log_var, d_X_rec, mse)

                    # 4.2. the loss of latent spaces distances:
                    #     - distances of drugs in the cluster to the cluster centroid 
                    #     + distances of drugs outside the cluster to the cluster centroid 
                    D_latent_dist_loss = cluster_mu_distance(d_mu, sensitive)
                    
                    # adding all up
                    if D_kld_weight is None:
                        D_kld_weight = batch_size / dataset_sizes[phase]

                    D_VAE_loss = D_recon_loss_weight * D_recon_loss + D_kld_weight * D_kld + D_cluster_distance_weight * D_latent_dist_loss # + D_update_ratio_weight * D_overlap_loss


                    # 5 requiring the updated cluster overlaping with the old clustering
                    d_name_dt = d_name.detach().numpy().astype(int)
                    d_name_uq = np.sort(np.unique(d_name_dt))
                    d_sens_k = torch.zeros(n_drugs)
                    for d in d_name_uq:
                        d_sens_k[d] = torch.mean(y_hat[d_name_dt == d]) > sens_cutoff

                    if len(set(d_names_k_old)) > 0:
                        d_overlap_ratio = torch.sum(d_sens_k[d_names_k_old]) / sum([d in d_name_uq for d in d_names_k_old])
                        D_overlap_loss = - d_overlap_ratio # at least 50% overlapping, the more overlap the better?
                    else:
                        D_overlap_loss = 0
                
                else:
                    D_VAE_loss = 0
                    D_overlap_loss = 0

                
                # Add up three losses:
                loss =  C_VAE_loss_weight * C_VAE_loss + D_VAE_loss_weight * D_VAE_loss + predict_loss_weight * prediction_loss + C_update_ratio_weight * C_overlap_loss + D_update_ratio_weight * D_overlap_loss
                # if C_VAE_loss_weight > 0:
                #     print(f'     C_VAE_loss: {C_VAE_loss}')
                #     print(f"   c: {C_VAE_loss_weight * C_VAE_loss}")
                #     print(f'     D_VAE_loss: {D_VAE_loss}')
                #     print(f"   d: {D_VAE_loss_weight * D_VAE_loss}")
                #     print(f'     prediction_loss: {prediction_loss}')
                #     print(f"   p: {predict_loss_weight * prediction_loss}")
                #     print(f'     C_overlap_loss: {C_overlap_loss}')
                #     print(f"   c_o: {C_update_ratio_weight * C_overlap_loss}")
                #     print(f'     D_overlap_loss: {D_overlap_loss}')
                #     print(f"   d_o: {D_update_ratio_weight * D_overlap_loss}")
                #     print(f'   Total loss: {loss}')

                optimizer.zero_grad()

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    # addressing the instability by lowering the learning rate or use gradient clipping
                    # torch.nn.utils.clip_grad_norm_(d_vae_predictor.parameters(), 0.01)
                    # update the weights
                    optimizer.step()

                # print loss statistics
                running_loss += loss.item()
                running_prediction_loss += (predict_loss_weight * prediction_loss).item()   
                running_c_update_overlap += (C_update_ratio_weight * C_overlap_loss)
                running_d_update_overlap += (D_update_ratio_weight * D_overlap_loss)
                # print(f"   running_d_update_overlap: {running_d_update_overlap}")

                if C_VAE_loss_weight > 0:
                    running_c_vae_loss += (C_VAE_loss_weight * C_VAE_loss).item()
                    running_c_recon_loss += (C_VAE_loss_weight * C_recon_loss_weight * C_recon_loss).item()
                    running_c_kld_loss += (C_VAE_loss_weight * C_kld_weight * C_kld).item()
                    running_c_cluster_d += (C_VAE_loss_weight * C_cluster_distance_weight * C_latent_dist_loss).item()
                    
                if D_VAE_loss_weight > 0:            
                    running_d_vae_loss += (D_VAE_loss_weight * D_VAE_loss).item()
                    running_d_recon_loss += (D_VAE_loss_weight * D_recon_loss_weight * D_recon_loss).item()
                    running_d_kld_loss += (D_VAE_loss_weight * D_kld_weight * D_kld).item()
                    running_d_cluster_d += (D_VAE_loss_weight * D_cluster_distance_weight * D_latent_dist_loss).item()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_prediction_loss = running_prediction_loss / dataset_sizes[phase]
            epoch_c_update_overlap = running_c_update_overlap / (batchidx + 1)
            epoch_d_update_overlap = running_d_update_overlap / (batchidx + 1)

            epoch_c_vae_loss = running_c_vae_loss / dataset_sizes[phase]
            epoch_c_recon_loss = running_c_recon_loss / dataset_sizes[phase]
            epoch_c_kld_loss = running_c_kld_loss / dataset_sizes[phase]
            epoch_c_cluster_d = running_c_cluster_d / dataset_sizes[phase]

            epoch_d_vae_loss = running_d_vae_loss / dataset_sizes[phase]
            epoch_d_recon_loss = running_d_recon_loss / dataset_sizes[phase]
            epoch_d_kld_loss = running_d_kld_loss / dataset_sizes[phase]
            epoch_d_cluster_d = running_d_cluster_d / dataset_sizes[phase]
            
            #if phase == 'train':
            #    scheduler.step(epoch_loss)
                
            # last_lr = scheduler.optimizer.param_groups[0]['lr']
            loss_hist[epoch,phase] = epoch_loss
            c_vae_loss_hist[epoch,phase] = epoch_c_vae_loss
            c_recon_loss_hist[epoch,phase] = epoch_c_recon_loss
            c_kld_hist[epoch,phase] = epoch_c_kld_loss
            c_cluster_dist_hist[epoch,phase] = epoch_c_cluster_d
            c_update_overlap_hist[epoch,phase] = epoch_c_update_overlap
            d_vae_loss_hist[epoch,phase] = epoch_d_vae_loss
            d_recon_loss_hist[epoch,phase] = epoch_d_recon_loss
            d_kld_hist[epoch,phase] = epoch_d_kld_loss
            d_cluster_dist_hist[epoch,phase] = epoch_d_cluster_d
            d_update_overlap_hist[epoch,phase] = epoch_d_update_overlap
            prediction_loss_hist[epoch,phase] = epoch_prediction_loss
            
            if phase == 'val' and epoch >= 5 and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch

                if best_model_cache == "memory":
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    torch.save(model.state_dict(), save_path+"_bestcahce.pkl")
            elif phase == 'val' and best_loss == np.inf and epoch == round(n_epochs/2):
                if best_model_cache == "memory":
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    print(f'          save model half way (epoch {epoch}) since testing loss is NaN')
                    torch.save(model.state_dict(), save_path+"_bestcahce.pkl")
                    best_epoch = epoch
                
    train_hist = [loss_hist, c_vae_loss_hist, c_recon_loss_hist, c_kld_hist, c_cluster_dist_hist, c_update_overlap_hist, d_vae_loss_hist, d_recon_loss_hist, d_kld_hist, d_cluster_dist_hist, d_update_overlap_hist, prediction_loss_hist]

    # Select best model wts if use memory to cahce models
    if best_model_cache == "memory":
        torch.save(best_model_wts, save_path)
        model.load_state_dict(best_model_wts)  
    else:
        print(f'            Best epoc with test loss: epoch {best_epoch}')
        model.load_state_dict((torch.load(save_path+"_bestcahce.pkl")))
        torch.save(model.state_dict(), save_path)


    print('NUMBER OF CALL TO DRUG SENS CUTOFF FNCT: ' + str(call_counter))

    return model, train_hist, best_epoch

def train_dvae_w_args():
    train_CDPmodel_local(
        model = d_vae_model_dict['model'], 
        data_loaders = d_vae_model_dict['data_loaders'], 
        c_names_k_old = d_vae_model_dict['c_names_k_old'], 
        d_names_k_old = d_vae_model_dict['d_names_k_old'], 
        C_VAE_loss_weight = d_vae_model_dict['C_VAE_loss_weight'], 
        C_recon_loss_weight = d_vae_model_dict['C_recon_loss_weight'], 
        C_kld_weight = d_vae_model_dict['C_kld_weight'],
        C_cluster_distance_weight = d_vae_model_dict['C_cluster_distance_weight'],
        C_update_ratio_weight = d_vae_model_dict['C_update_ratio_weight'],
        D_VAE_loss_weight = d_vae_model_dict['D_VAE_loss_weight'], 
        D_recon_loss_weight = d_vae_model_dict['D_recon_loss_weight'],  
        D_kld_weight = d_vae_model_dict['D_kld_weight'],  
        D_cluster_distance_weight = d_vae_model_dict['D_cluster_distance_weight'],  
        D_update_ratio_weight = d_vae_model_dict['D_update_ratio_weight'],
        predict_loss_weight = d_vae_model_dict['predict_loss_weight'],
        sens_cutoff = d_vae_model_dict['sens_cutoff'],
        optimizer = d_vae_model_dict['optimizer'], 
        n_epochs = d_vae_model_dict['n_epochs'], 
        scheduler = d_vae_model_dict['scheduler'],
        save_path = d_vae_model_dict['save_path']
    )    


def train_cvae_w_args():
    train_CDPmodel_local(
        model = c_vae_model_dict['model'], 
        data_loaders = c_vae_model_dict['data_loaders'], 
        c_names_k_old = c_vae_model_dict['c_names_k_old'], 
        d_names_k_old = c_vae_model_dict['d_names_k_old'], 
        C_VAE_loss_weight = c_vae_model_dict['C_VAE_loss_weight'], 
        C_recon_loss_weight = c_vae_model_dict['C_recon_loss_weight'], 
        C_kld_weight = c_vae_model_dict['C_kld_weight'],
        C_cluster_distance_weight = c_vae_model_dict['C_cluster_distance_weight'],
        C_update_ratio_weight = c_vae_model_dict['C_update_ratio_weight'],
        D_VAE_loss_weight = c_vae_model_dict['D_VAE_loss_weight'], 
        D_recon_loss_weight = c_vae_model_dict['D_recon_loss_weight'],  
        D_kld_weight = c_vae_model_dict['D_kld_weight'],  
        D_cluster_distance_weight = c_vae_model_dict['D_cluster_distance_weight'],  
        D_update_ratio_weight = c_vae_model_dict['D_update_ratio_weight'],
        predict_loss_weight = c_vae_model_dict['predict_loss_weight'],
        sens_cutoff = c_vae_model_dict['sens_cutoff'],
        optimizer = c_vae_model_dict['optimizer'], 
        n_epochs = c_vae_model_dict['n_epochs'], 
        scheduler = c_vae_model_dict['scheduler'],
        save_path = c_vae_model_dict['save_path']
    )    


In [84]:
## Confirm data loader has redundancies:
print(c_train.shape)
print(d_data.shape)

np.sum(~np.isnan(cdr_train.to_numpy())) / 50 
## doesn't explain the total number of batches. in fact we would still only be iterating through the active members of a bicluster.

(117, 355)
(30, 150)


66.96